In [13]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Define the path to your dataset
data_path = 'D:/Gerina Mary C/gtzan_dataset/Data/genres_original/'

# Get the list of genres from the directory names
genres = os.listdir(data_path)

# Initialize lists to hold features and labels
data = []
labels = []

# Iterate over each genre
for genre in genres:
    # Get the list of song files for this genre
    song_files = os.listdir(os.path.join(data_path, genre))
    
    # Iterate over each song file
    for song in song_files:
        # Load the song file using librosa
        y, sr = librosa.load(os.path.join(data_path, genre, song))
        
        # Extract features from the audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        
        # Append the mean of the features and the genre label to the respective lists
        data.append(np.mean(mfcc, axis=1))
        labels.append(genre)

# Convert features and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the data for the CRNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


# Define CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(genres), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=42, validation_data=(X_test, y_test))

# Extract features from CNN intermediate layer
cnn_model = Model(inputs=model.input, outputs=model.get_layer(index=3).output)
X_train_cnn = cnn_model.predict(X_train)
X_test_cnn = cnn_model.predict(X_test)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train_cnn, y_train)

# Evaluate combined CNN+KNN model
accuracy = knn_classifier.score(X_test_cnn, y_test)
print("Accuracy of combined CNN+KNN model:", accuracy)

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 4.4118 - accuracy: 0.2037 - val_loss: 2.3560 - val_accuracy: 0.2900
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 1.9038 - accuracy: 0.4263 - val_loss: 2.2840 - val_accuracy: 0.3200
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 1.6196 - accuracy: 0.4550 - val_loss: 1.8701 - val_accuracy: 0.3950
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 1.6005 - accuracy: 0.4775 - val_loss: 1.8451 - val_accuracy: 0.4350
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 1.4221 - accuracy: 0.5150 - val_loss: 2.0641 - val_accuracy: 0.3250
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 1.4527 - accuracy: 0.5263 - val_loss: 1.6245 - val_accuracy: 0.4550
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 1.2989 - accuracy: 0.5400 - val_loss: 1.6350 - val_accuracy: 0.4300
Epoch 8/100


Epoch 59/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5606 - accuracy: 0.8075 - val_loss: 1.5341 - val_accuracy: 0.5750
Epoch 60/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4485 - accuracy: 0.8512 - val_loss: 1.6354 - val_accuracy: 0.5800
Epoch 61/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4451 - accuracy: 0.8550 - val_loss: 1.6324 - val_accuracy: 0.6000
Epoch 62/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5046 - accuracy: 0.8425 - val_loss: 1.7066 - val_accuracy: 0.5950
Epoch 63/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5984 - accuracy: 0.7825 - val_loss: 1.9526 - val_accuracy: 0.5250
Epoch 64/100
20/20 [==============================] - 0s 4ms/step - loss: 0.9606 - accuracy: 0.7050 - val_loss: 1.6675 - val_accuracy: 0.5300
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6052 - accuracy: 0.7700 - val_loss: 1.5133 - val_accuracy: 0.6300
Epoch 